In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-jan-2022/sample_submission.csv
/kaggle/input/tabular-playground-series-jan-2022/train.csv
/kaggle/input/tabular-playground-series-jan-2022/test.csv


# Basic EDA

In [2]:
train = pd.read_csv('../input/tabular-playground-series-jan-2022/train.csv')
test = pd.read_csv('../input/tabular-playground-series-jan-2022/test.csv')
sample_submit = pd.read_csv('../input/tabular-playground-series-jan-2022/sample_submission.csv')

In [3]:
train.head()

,row_id,date,country,store,product,num_sold
0,0,2015-01-01,Finland,KaggleMart,Kaggle Mug,329
1,1,2015-01-01,Finland,KaggleMart,Kaggle Hat,520
2,2,2015-01-01,Finland,KaggleMart,Kaggle Sticker,146
3,3,2015-01-01,Finland,KaggleRama,Kaggle Mug,572
4,4,2015-01-01,Finland,KaggleRama,Kaggle Hat,911


In [4]:
train.shape

(26298, 6)

In [5]:
train.isnull().sum()

row_id      0
date        0
country     0
store       0
product     0
num_sold    0
dtype: int64

In [6]:
train['country'].value_counts()

Finland    8766
Norway     8766
Sweden     8766
Name: country, dtype: int64

In [7]:
train['store'].value_counts()

KaggleMart    13149
KaggleRama    13149
Name: store, dtype: int64

In [8]:
train['product'].value_counts()

Kaggle Mug        8766
Kaggle Hat        8766
Kaggle Sticker    8766
Name: product, dtype: int64

In [9]:
train.describe()

,row_id,num_sold
count,26298.000000,26298.000000
mean,13148.500000,387.533577
std,7591.723026,266.076193
min,0.000000,70.000000
25%,6574.250000,190.000000
50%,13148.500000,315.000000
75%,19722.750000,510.000000
max,26297.000000,2884.000000


In [10]:
test.head()

,row_id,date,country,store,product
0,26298,2019-01-01,Finland,KaggleMart,Kaggle Mug
1,26299,2019-01-01,Finland,KaggleMart,Kaggle Hat
2,26300,2019-01-01,Finland,KaggleMart,Kaggle Sticker
3,26301,2019-01-01,Finland,KaggleRama,Kaggle Mug
4,26302,2019-01-01,Finland,KaggleRama,Kaggle Hat


In [11]:
test.shape

(6570, 5)

In [12]:
test.isnull().sum()

row_id     0
date       0
country    0
store      0
product    0
dtype: int64

In [13]:
test['country'].value_counts()

Finland    2190
Norway     2190
Sweden     2190
Name: country, dtype: int64

In [14]:
test['store'].value_counts()

KaggleMart    3285
KaggleRama    3285
Name: store, dtype: int64

In [15]:
test['product'].value_counts()

Kaggle Mug        2190
Kaggle Hat        2190
Kaggle Sticker    2190
Name: product, dtype: int64

In [16]:
test.describe()

,row_id
count,6570.000000
mean,29582.500000
std,1896.739966
min,26298.000000
25%,27940.250000
50%,29582.500000
75%,31224.750000
max,32867.000000


# Feature Engineering

In [17]:
data = pd.concat([train, test], axis=0, ignore_index=True)
data

,row_id,date,country,store,product,num_sold
0,0,2015-01-01,Finland,KaggleMart,Kaggle Mug,329.0
1,1,2015-01-01,Finland,KaggleMart,Kaggle Hat,520.0
2,2,2015-01-01,Finland,KaggleMart,Kaggle Sticker,146.0
3,3,2015-01-01,Finland,KaggleRama,Kaggle Mug,572.0
4,4,2015-01-01,Finland,KaggleRama,Kaggle Hat,911.0
...,...,...,...,...,...,...
32863,32863,2019-12-31,Sweden,KaggleMart,Kaggle Hat,NaN
32864,32864,2019-12-31,Sweden,KaggleMart,Kaggle Sticker,NaN
32865,32865,2019-12-31,Sweden,KaggleRama,Kaggle Mug,NaN
32866,32866,2019-12-31,Sweden,KaggleRama,Kaggle Hat,NaN


In [18]:
from datetime import datetime, date

data['date'] = pd.to_datetime(data.date, format = '%Y/%m/%d')
data

,row_id,date,country,store,product,num_sold
0,0,2015-01-01,Finland,KaggleMart,Kaggle Mug,329.0
1,1,2015-01-01,Finland,KaggleMart,Kaggle Hat,520.0
2,2,2015-01-01,Finland,KaggleMart,Kaggle Sticker,146.0
3,3,2015-01-01,Finland,KaggleRama,Kaggle Mug,572.0
4,4,2015-01-01,Finland,KaggleRama,Kaggle Hat,911.0
...,...,...,...,...,...,...
32863,32863,2019-12-31,Sweden,KaggleMart,Kaggle Hat,NaN
32864,32864,2019-12-31,Sweden,KaggleMart,Kaggle Sticker,NaN
32865,32865,2019-12-31,Sweden,KaggleRama,Kaggle Mug,NaN
32866,32866,2019-12-31,Sweden,KaggleRama,Kaggle Hat,NaN


In [19]:
import holidays

holiday_FI = holidays.CountryHoliday('FI', years=[2015, 2016, 2017, 2018, 2019])

holiday_FI

{datetime.date(2016, 1, 1): 'Uudenvuodenpäivä', datetime.date(2016, 1, 6): 'Loppiainen', datetime.date(2016, 3, 25): 'Pitkäperjantai', datetime.date(2016, 3, 27): 'Pääsiäispäivä', datetime.date(2016, 3, 28): '2. pääsiäispäivä', datetime.date(2016, 5, 1): 'Vappu', datetime.date(2016, 5, 5): 'Helatorstai', datetime.date(2016, 5, 15): 'Helluntaipäivä', datetime.date(2016, 6, 25): 'Juhannuspäivä', datetime.date(2016, 11, 5): 'Pyhäinpäivä', datetime.date(2016, 12, 6): 'Itsenäisyyspäivä', datetime.date(2016, 12, 25): 'Joulupäivä', datetime.date(2016, 12, 26): 'Tapaninpäivä', datetime.date(2016, 6, 24): 'Juhannusaatto', datetime.date(2016, 12, 24): 'Jouluaatto', datetime.date(2017, 1, 1): 'Uudenvuodenpäivä', datetime.date(2017, 1, 6): 'Loppiainen', datetime.date(2017, 4, 14): 'Pitkäperjantai', datetime.date(2017, 4, 16): 'Pääsiäispäivä', datetime.date(2017, 4, 17): '2. pääsiäispäivä', datetime.date(2017, 5, 1): 'Vappu', datetime.date(2017, 5, 25): 'Helatorstai', datetime.date(2017, 6, 4): 'He

In [20]:
holiday_NO = holidays.CountryHoliday('NO', years=[2015, 2016, 2017, 2018, 2019])
holiday_NO

{datetime.date(2016, 1, 1): 'Første nyttårsdag', datetime.date(2016, 5, 1): 'Arbeidernes dag', datetime.date(2016, 5, 17): 'Grunnlovsdag', datetime.date(2016, 12, 25): 'Første juledag', datetime.date(2016, 12, 26): 'Andre juledag', datetime.date(2016, 3, 24): 'Skjærtorsdag', datetime.date(2016, 3, 25): 'Langfredag', datetime.date(2016, 3, 27): 'Første påskedag', datetime.date(2016, 3, 28): 'Andre påskedag', datetime.date(2016, 5, 5): 'Kristi himmelfartsdag', datetime.date(2016, 5, 15): 'Første pinsedag', datetime.date(2016, 5, 16): 'Andre pinsedag', datetime.date(2017, 1, 1): 'Første nyttårsdag', datetime.date(2017, 5, 1): 'Arbeidernes dag', datetime.date(2017, 5, 17): 'Grunnlovsdag', datetime.date(2017, 12, 25): 'Første juledag', datetime.date(2017, 12, 26): 'Andre juledag', datetime.date(2017, 4, 13): 'Skjærtorsdag', datetime.date(2017, 4, 14): 'Langfredag', datetime.date(2017, 4, 16): 'Første påskedag', datetime.date(2017, 4, 17): 'Andre påskedag', datetime.date(2017, 5, 25): 'Krist

In [21]:
holiday_SE = holidays.CountryHoliday('SE', years=[2015, 2016, 2017, 2018, 2019])
holiday_SE

{datetime.date(2016, 1, 3): 'Söndag', datetime.date(2016, 1, 10): 'Söndag', datetime.date(2016, 1, 17): 'Söndag', datetime.date(2016, 1, 24): 'Söndag', datetime.date(2016, 1, 31): 'Söndag', datetime.date(2016, 2, 7): 'Söndag', datetime.date(2016, 2, 14): 'Söndag', datetime.date(2016, 2, 21): 'Söndag', datetime.date(2016, 2, 28): 'Söndag', datetime.date(2016, 3, 6): 'Söndag', datetime.date(2016, 3, 13): 'Söndag', datetime.date(2016, 3, 20): 'Söndag', datetime.date(2016, 3, 27): 'Påskdagen, Söndag', datetime.date(2016, 4, 3): 'Söndag', datetime.date(2016, 4, 10): 'Söndag', datetime.date(2016, 4, 17): 'Söndag', datetime.date(2016, 4, 24): 'Söndag', datetime.date(2016, 5, 1): 'Första maj, Söndag', datetime.date(2016, 5, 8): 'Söndag', datetime.date(2016, 5, 15): 'Pingstdagen, Söndag', datetime.date(2016, 5, 22): 'Söndag', datetime.date(2016, 5, 29): 'Söndag', datetime.date(2016, 6, 5): 'Söndag', datetime.date(2016, 6, 12): 'Söndag', datetime.date(2016, 6, 19): 'Söndag', datetime.date(2016, 

In [22]:
holiday_dict = holiday_FI.copy()
holiday_dict.update(holiday_NO)
holiday_dict.update(holiday_SE)

holiday_dict

{datetime.date(2016, 1, 1): 'Nyårsdagen',
 datetime.date(2016, 1, 6): 'Trettondedag jul',
 datetime.date(2016, 3, 25): 'Långfredagen',
 datetime.date(2016, 3, 27): 'Påskdagen, Söndag',
 datetime.date(2016, 3, 28): 'Annandag påsk',
 datetime.date(2016, 5, 1): 'Första maj, Söndag',
 datetime.date(2016, 5, 5): 'Kristi himmelsfärdsdag',
 datetime.date(2016, 5, 15): 'Pingstdagen, Söndag',
 datetime.date(2016, 6, 25): 'Midsommardagen',
 datetime.date(2016, 11, 5): 'Alla helgons dag',
 datetime.date(2016, 12, 6): 'Itsenäisyyspäivä',
 datetime.date(2016, 12, 25): 'Juldagen, Söndag',
 datetime.date(2016, 12, 26): 'Annandag jul',
 datetime.date(2016, 6, 24): 'Midsommarafton',
 datetime.date(2016, 12, 24): 'Julafton',
 datetime.date(2017, 1, 1): 'Nyårsdagen, Söndag',
 datetime.date(2017, 1, 6): 'Trettondedag jul',
 datetime.date(2017, 4, 14): 'Långfredagen',
 datetime.date(2017, 4, 16): 'Påskdagen, Söndag',
 datetime.date(2017, 4, 17): 'Annandag påsk',
 datetime.date(2017, 5, 1): 'Första maj',
 d

In [23]:
data['date'] = pd.to_datetime(data['date']) 
data['holiday_name'] = data['date'].map(holiday_dict)
data['is_holiday'] = np.where(data['holiday_name'].notnull(), 1, 0)
data

,row_id,date,country,store,product,num_sold,holiday_name,is_holiday
0,0,2015-01-01,Finland,KaggleMart,Kaggle Mug,329.0,Nyårsdagen,1
1,1,2015-01-01,Finland,KaggleMart,Kaggle Hat,520.0,Nyårsdagen,1
2,2,2015-01-01,Finland,KaggleMart,Kaggle Sticker,146.0,Nyårsdagen,1
3,3,2015-01-01,Finland,KaggleRama,Kaggle Mug,572.0,Nyårsdagen,1
4,4,2015-01-01,Finland,KaggleRama,Kaggle Hat,911.0,Nyårsdagen,1
...,...,...,...,...,...,...,...,...
32863,32863,2019-12-31,Sweden,KaggleMart,Kaggle Hat,NaN,Nyårsafton,1
32864,32864,2019-12-31,Sweden,KaggleMart,Kaggle Sticker,NaN,Nyårsafton,1
32865,32865,2019-12-31,Sweden,KaggleRama,Kaggle Mug,NaN,Nyårsafton,1
32866,32866,2019-12-31,Sweden,KaggleRama,Kaggle Hat,NaN,Nyårsafton,1
